In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [54]:
df = pd.read_csv('data/dat_train2.csv')

In [55]:
# split id into two columns, customer_id and account_id
df['customer_id'] = df['id'].apply(lambda x: x.split(' ')[0])
df['account_id'] = df['id'].apply(lambda x: x.split(' ')[1])

# reformat dates
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'])

In [56]:
# get unique combinations of ed_id and event_name
df[['ed_id', 'event_name']].drop_duplicates().sort_values(by=['ed_id', 'event_name'])

,ed_id,event_name
68,1,promotion_created
15,2,campaign_click
14,3,application_web_submit
16,4,browse_products
36,5,view_cart
318,6,begin_checkout
407,7,place_order_web
659,8,place_downpayment
23319939,9,customer_requested_catalog_(digital)
188507,10,fingerhut_university


In [57]:
df.head()

,ed_id,event_name,event_timestamp,journey_steps_until_end,id,sep,customer_id,account_id
0,12,application_web_approved,2023-03-22 08:45:22+00:00,1,-784961211 1773350293,-,-784961211,1773350293
1,19,application_web_view,2023-03-22 13:32:10+00:00,2,-784961211 1773350293,-,-784961211,1773350293
2,19,application_web_view,2023-03-22 13:32:10+00:00,3,-784961211 1773350293,-,-784961211,1773350293
3,19,application_web_view,2023-03-22 13:32:10+00:00,4,-784961211 1773350293,-,-784961211,1773350293
4,19,application_web_view,2023-03-22 13:32:10+00:00,5,-784961211 1773350293,-,-784961211,1773350293


In [58]:
df_clean = df.drop(columns=['event_name', 'id', 'sep'])

In [59]:
duplicate_mask = df_clean.duplicated(subset=['ed_id', 'event_timestamp', 'customer_id', 'account_id'])

In [60]:
df_clean = df_clean[~duplicate_mask].copy()

In [61]:
df_clean = df_clean.sort_values(by=['customer_id', 'event_timestamp'])

In [62]:
df_clean.head()

,ed_id,event_timestamp,journey_steps_until_end,customer_id,account_id
22761983,12,2022-10-31 13:45:59+00:00,1,-1000001271,551641434
22761984,4,2022-10-31 13:47:12+00:00,2,-1000001271,551641434
22761985,19,2022-10-31 13:47:36+00:00,3,-1000001271,551641434
22761986,19,2022-10-31 13:47:39+00:00,4,-1000001271,551641434
22761987,19,2022-10-31 13:47:41+00:00,5,-1000001271,551641434


In [63]:
df_clean['time_since_last_event'] = df_clean.groupby('customer_id')['event_timestamp'].diff()
df_clean['is_new_journey'] = (
    (df_clean['time_since_last_event'] > pd.Timedelta(days=60)) |
    (df_clean['time_since_last_event'].isnull())
).astype(int)
df_clean['journey_number'] = df_clean.groupby('customer_id')['is_new_journey'].cumsum()
df_clean['journey_id'] = df_clean['customer_id'].astype(str) + "_" + df_clean['journey_number'].astype(str)
df_clean = df_clean.drop(columns=['is_new_journey', 'journey_number', 'time_since_last_event', 'journey_steps_until_end'])

In [64]:
df_clean.reset_index(drop=True, inplace=True)

In [66]:
# fix step counts and add target variable

df_clean['journey_step'] = df_clean.groupby('journey_id').cumcount() + 1
successful_journeys = df_clean[df_clean['ed_id'] == 28]['journey_id'].unique()
df_clean['target'] = df_clean['journey_id'].isin(successful_journeys).astype(int)

In [68]:
df = df_clean.copy()

In [69]:
df.head()

,ed_id,event_timestamp,customer_id,account_id,journey_id,journey_step,target
0,12,2022-10-31 13:45:59+00:00,-1000001271,551641434,-1000001271_1,1,0
1,4,2022-10-31 13:47:12+00:00,-1000001271,551641434,-1000001271_1,2,0
2,19,2022-10-31 13:47:36+00:00,-1000001271,551641434,-1000001271_1,3,0
3,19,2022-10-31 13:47:39+00:00,-1000001271,551641434,-1000001271_1,4,0
4,19,2022-10-31 13:47:41+00:00,-1000001271,551641434,-1000001271_1,5,0


In [70]:
# drop unfinished journeys

max_dataset_date = df['event_timestamp'].max()
last_event_per_journey = df.groupby('journey_id')['event_timestamp'].max()
definiteively_lapsed = last_event_per_journey[
    last_event_per_journey < (max_dataset_date - pd.Timedelta(days=60))
].index

df = df[
    (df['target'] == 1) |
    (df['journey_id'].isin(definiteively_lapsed))
].copy()

In [73]:
# apply the random cutoff

df_with_target = df.copy()
df = df[df['ed_id'] != 28].copy()
journey_lengths = df.groupby('journey_id').size().reset_index(name='max_steps')

In [75]:
journey_lengths['cutoff_step'] = journey_lengths['max_steps'].apply(
    lambda x: np.random.randint(1, x + 1) if x > 1 else 1
)

df_cutoff = df.merge(journey_lengths[['journey_id', 'cutoff_step']], on='journey_id')
df_cutoff = df_cutoff[df_cutoff['journey_step'] <= df_cutoff['cutoff_step']].copy()
df_cutoff = df_cutoff.drop(columns=['cutoff_step'])

In [78]:
df_cutoff.head()

,ed_id,event_timestamp,customer_id,account_id,journey_id,journey_step,target
0,12,2022-10-31 13:45:59+00:00,-1000001271,551641434,-1000001271_1,1,0
1,4,2022-10-31 13:47:12+00:00,-1000001271,551641434,-1000001271_1,2,0
7,12,2021-11-11 01:30:43+00:00,-100000293,92584960,-100000293_1,1,0
8,4,2021-11-11 01:39:24+00:00,-100000293,92584960,-100000293_1,2,0
9,4,2021-11-11 01:40:31+00:00,-100000293,92584960,-100000293_1,3,0


In [80]:
# flatten and engineer features
event_dummies = pd.get_dummies(df_cutoff['ed_id'], prefix='ed_id')
df_with_dummies = pd.concat([df_cutoff[['journey_id', 'event_timestamp', 'target']], event_dummies], axis=1)

df_model = df_with_dummies.groupby('journey_id').agg(
    target=('target', 'max'),
    total_events=('journey_id', 'count'),
    journey_start=('event_timestamp', 'min'),
    journey_end=('event_timestamp', 'max'),

    count_app_approved=('ed_id_12', 'sum'),
    count_app_declined=('ed_id_13', 'sum'),
    count_browse=('ed_id_4', 'sum'),
    count_checkout=('ed_id_6', 'sum')
)

df_model['duration_hours'] = (df_model['journey_end'] - df_model['journey_start']).dt.total_seconds() / 3600
df_model['events_per_hour'] = df_model['total_events'] / (df_model['duration_hours'] + 1e-6)

df_model = df_model.drop(columns=['journey_start', 'journey_end'])

In [81]:
df_model.head()

,target,total_events,count_app_approved,count_app_declined,count_browse,count_checkout,duration_hours,events_per_hour
journey_id,,,,,,,,
-1000001271_1,0,2,1,0,1,0,0.020278,98.625273
-100000293_1,0,20,1,0,11,1,43.146389,0.463538
-100001025_1,0,7,0,0,0,0,7.183611,0.974440
-1000011207_1,0,10,1,0,4,0,0.213333,46.874780
-100001164_1,0,1,0,0,0,0,0.000000,1000000.000000


In [84]:
df_model.reset_index(inplace=True)

In [ ]:
df_model.head()

,journey_id,target,total_events,count_app_approved,count_app_declined,count_browse,count_checkout,duration_hours,events_per_hour
0,-1000001271_1,0,2,1,0,1,0,0.020278,98.625273
1,-100000293_1,0,20,1,0,11,1,43.146389,0.463538
2,-100001025_1,0,7,0,0,0,0,7.183611,0.974440
3,-1000011207_1,0,10,1,0,4,0,0.213333,46.874780
4,-100001164_1,0,1,0,0,0,0,0.000000,1000000.000000


In [88]:
df_model.shape

(2090818, 9)